In [88]:
# api_key = "PKABTN0SFPC9AF832CW4"
# secret_key = "t8unjLT9hjtvtC23uTxrL1FEm1l6alM1wDpsCpYc"

api_key = "PKWUVJ0QKZP1BISFGU7Y"
secret_key = "KzTwX4H0JK6LaVkcVXJ1KMomHGIMTGM0HyRgwwK6"

#### We use paper environment for this example ####
paper = True # Please do not modify this. This example is for paper trading only.
####

# Below are the variables for development this documents
# Please do not change these variables
trade_api_url = None
trade_api_wss = None
data_api_url = None
stream_data_wss = None

In [89]:
import alpaca_trade_api as tradeapi

BASE_URL = "https://paper-api.alpaca.markets"

# Initialize API
api = tradeapi.REST(api_key, secret_key, BASE_URL, api_version="v2")

# Close all open positions
try:
    api.close_all_positions()
    print("Closed all positions successfully.")
except Exception as e:
    print(f"Error closing positions: {e}")

# Cancel all open orders
try:
    api.cancel_all_orders()
    print("Canceled all open orders successfully.")
except Exception as e:
    print(f"Error canceling orders: {e}")

# Reset cash balance (Alpaca does not provide an API method for this)
print("To reset cash balance, manually adjust it in the Alpaca dashboard.")

Closed all positions successfully.
Canceled all open orders successfully.
To reset cash balance, manually adjust it in the Alpaca dashboard.


In [90]:
import os

if api_key is None:
    api_key = os.environ.get('ALPACA_API_KEY')

if secret_key is None:
    secret_key = os.environ.get('ALPACA_SECRET_KEY')

In [91]:
# install alpaca-py if it is not available
try:
    import alpaca
except ImportError:
    !python3 -m pip install alpaca-py
    import alpaca

In [92]:
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

from alpaca.trading.client import TradingClient
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.historical.corporate_actions import CorporateActionsClient
from alpaca.data.historical.stock import StockHistoricalDataClient
from alpaca.trading.stream import TradingStream
from alpaca.data.live.stock import StockDataStream

from alpaca.data.requests import (
    CorporateActionsRequest,
    StockBarsRequest,
    StockQuotesRequest,
    StockTradesRequest,
)
from alpaca.trading.requests import (
    ClosePositionRequest,
    GetAssetsRequest,
    GetOrdersRequest,
    LimitOrderRequest,
    MarketOrderRequest,
    StopLimitOrderRequest,
    StopLossRequest,
    StopOrderRequest,
    TakeProfitRequest,
    TrailingStopOrderRequest,
)
from alpaca.trading.enums import (
    AssetExchange,
    AssetStatus,
    OrderClass,
    OrderSide,
    OrderType,
    QueryOrderStatus,
    TimeInForce,
)

In [93]:
# to run async code in jupyter notebook
import nest_asyncio
nest_asyncio.apply()

In [94]:
# setup clients
trade_client = TradingClient(api_key=api_key, secret_key=secret_key, paper=paper, url_override=trade_api_url)

In [95]:
# check trading account
# You can check definition of each field in the following documents
# ref. https://docs.alpaca.markets/docs/account-plans
# ref. https://docs.alpaca.markets/reference/getaccount-1
acct = trade_client.get_account()
acct_config = trade_client.get_account_configurations()

In [96]:
# set account configuration
# ref. https://docs.alpaca.markets/reference/patchaccountconfig-1
req = acct_config
req.fractional_trading = not req.fractional_trading # toggle fractional trading
acct_config_new = trade_client.set_account_configurations(req)

# revert changes
req = acct_config_new
req.fractional_trading = not req.fractional_trading # toggle fractional trading
acct_config_reverted = trade_client.set_account_configurations(req)

In [97]:
# get list of assets which are us_equity (default), active, and in NASDAQ
# ref. https://docs.alpaca.markets/reference/get-v2-assets-1
req = GetAssetsRequest(
  # asset_class=AssetClass.US_EQUITY,  # default asset_class is us_equity
  status=AssetStatus.ACTIVE,
  exchange=AssetExchange.NASDAQ,
)
assets = trade_client.get_all_assets(req)

# Trading Strategy

In [98]:
import os
import pickle
import time
import schedule
import pandas as pd
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
import numpy as np


# # Stocks for Trading & Tracking： we have different strategies on etfs and stocks
# stocks = {
#     "NVDA": "/Users/dingyiling/Desktop/RealTime_Group5_TradingCompetition/lstm_local_extremaNew_NVDA.h5",
#     "C":    "/Users/dingyiling/Desktop/RealTime_Group5_TradingCompetition/lstm_local_extrema_C.h5",
#     "SBUX": "/Users/dingyiling/Desktop/RealTime_Group5_TradingCompetition/lstm_local_extremaNew_SBUX.h5"
# }
# etfs = ["SPY", "VIS"]

# print(symbols )

## Fetch the real time data

In [99]:
import logging
import traceback
from requests.exceptions import RequestException

# Fetching and Saving Stock Data
def get_real_time_market_data(symbol):
    now = datetime.now(ZoneInfo("America/New_York"))
    try:
        req = StockBarsRequest(
            symbol_or_symbols=[symbol],
            timeframe=TimeFrame(amount=1, unit=TimeFrameUnit.Minute),
            start=now - timedelta(days=7)
        )
            
        df = stock_historical_data_client.get_stock_bars(req).df
        return df
    except Exception as e:
        print(f"Failed to fetch real-time data for {symbol}: {e}")
        return None
        
logging.basicConfig(
    filename="market_data.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

from alpaca.data.historical.stock import StockHistoricalDataClient

stock_historical_data_client = StockHistoricalDataClient(api_key, secret_key, url_override = data_api_url)

# def fetch_and_save_data():
#     now = datetime.now(ZoneInfo("America/New_York"))
#     for symbol in symbols:
#         try:
#             req = StockBarsRequest(
#                 symbol_or_symbols=[symbol],
#                 timeframe=TimeFrame(amount=1, unit=TimeFrameUnit.Minute),
#                 start=now - timedelta(days=7)
#             )
            
#             df = stock_historical_data_client.get_stock_bars(req).df

#             # Define file paths
#             pickle_file_path = os.path.join("/Users/dingyiling/Desktop/RealTime_Group5_TradingCompetition/", f"{symbol}.pkl")
#             csv_file_path = os.path.join("/Users/dingyiling/Desktop/RealTime_Group5_TradingCompetition/", f"{symbol}.csv")

#             # Save to Pickle
#             with open(pickle_file_path, "wb") as f:
#                 pickle.dump(df, f)

#             # Save to CSV
#             df.to_csv(csv_file_path, index=True)

#             logging.info(f"Updated {symbol}.csv and {symbol}.pkl at {datetime.now()}")

#         except Exception as e:
#             logging.error(f"Error updating {symbol}: {e}")
#             logging.error(traceback.format_exc())



In [100]:
# Place Orders
def place_order(symbol, qty, side):
    if qty <= 0:
        print(f"{symbol} - Order not placed. Quantity is zero or negative.")
        return
    try:
        api.submit_order(
            symbol=symbol,
            qty=qty,
            side=side,
            type="market",
            time_in_force="gtc"
        )
        print(f"{symbol} - Order Placed: {side.upper()} {qty} shares")
    except Exception as e:
        print(f"{symbol} - Order Failed: {e}")

## ETFS Strategy

In [101]:
# ETF Trading Parameters
etfs_new=["CME","REGN"]
initial_buy_notional = 5000.0

rebuy_notional = 1000.0
threshold_sell_20 = 0.003
threshold_sell_40 = 0.004
threshold_sell_all = 0.005
threshold_rebuy = -0.005

holding_etf_new = {symbol: 0.0 for symbol in etfs_new}
last_buy_price = {}
cash_etf = {}

In [102]:
# ETF Trading Logic
def execute_etf_trading_new():
    global last_buy_price, holding_etf_new, cash_etf

    now = datetime.now(ZoneInfo("America/New_York"))

    # Ensure ETF trading only happens during market hours (9:30 AM - 4:00 PM ET)
    # if now.hour < 9 or now.hour >= 16:
    #     return  

    for symbol in etfs_new:
        df = get_real_time_market_data(symbol)
        if df is None or df.empty:
            continue

        # Get current price information
        market_open_price = df["open"].iloc[-1] if now.hour == 9 and now.minute == 30 else None
        current_price = df["close"].iloc[-1]

        # Initialize first purchase logic at market open
        if symbol not in last_buy_price:
            first_buy_price = market_open_price if market_open_price is not None else current_price
            last_buy_price[symbol] = first_buy_price  # Store first buy price
            holding_etf_new[symbol] = int(initial_buy_notional / first_buy_price)  # Ensure integer shares
            cash_etf[symbol] = -initial_buy_notional

            # Execute market order to buy ETFs
            if holding_etf_new[symbol] > 0:
                place_order(symbol, qty=holding_etf_new[symbol], side="buy")
                print(f"{symbol} First Buy: ${initial_buy_notional}, Price: {first_buy_price:.2f}")

        # Calculate price change from last purchase price
        
        price_change = (current_price / last_buy_price[symbol]) - 1

        # Selling Logic
        if holding_etf_new[symbol] > 0:
            if price_change >= threshold_sell_all:
                # Sell all holdings
                place_order(symbol, qty=holding_etf_new[symbol], side="sell")
                cash_etf[symbol] += holding_etf_new[symbol] * current_price
                print(f"Sold ALL {symbol} at {current_price:.2f}, Total Cash: {cash_etf[symbol]:.2f}")
                holding_etf_new[symbol] = 0
            elif price_change >= threshold_sell_40:
                # Sell 40% of holdings
                sell_qty = int(holding_etf_new[symbol] * 0.4)
                if sell_qty > 0:
                    place_order(symbol, qty=sell_qty, side="sell")
                    cash_etf[symbol] += sell_qty * current_price
                    holding_etf_new[symbol] -= sell_qty
                    print(f"Sold 40% {symbol} at {current_price:.2f}, Remaining: {holding_etf_new[symbol]} shares")
            elif price_change >= threshold_sell_20:
                # Sell 20% of holdings
                sell_qty = int(holding_etf_new[symbol] * 0.2)
                if sell_qty > 0:
                    place_order(symbol, qty=sell_qty, side="sell")
                    cash_etf[symbol] += sell_qty * current_price
                    holding_etf_new[symbol] -= sell_qty
                    print(f"Sold 20% {symbol} at {current_price:.2f}, Remaining: {holding_etf_new[symbol]} shares")

        # Rebuy Logic (if price drops 0.5% below the last purchase price)
        if holding_etf_new[symbol] == 0 and price_change <= threshold_rebuy:
            rebuy_shares = int(rebuy_notional / current_price)
            if rebuy_shares > 0:
                place_order(symbol, qty=rebuy_shares, side="buy")
                holding_etf_new[symbol] = rebuy_shares
                cash_etf[symbol] -= rebuy_notional
                last_buy_price[symbol] = current_price  # Update last purchase price
                print(f"Rebought {symbol}: {rebuy_shares} shares at {current_price:.2f}, Total Cash: {cash_etf[symbol]:.2f}")


In [103]:
etfs_VIS=["VIS"]
threshold_sell_20_VIS = 0.001
threshold_sell_40_VIS = 0.002
threshold_sell_all_VIS = 0.003
holding_etf_keep_VIS={symbol: 0.0 for symbol in etfs_VIS}

In [104]:
def execute_etf_trading_VIS():
    global last_buy_price, holding_etf_keep_VIS, cash_etf

    now = datetime.now(ZoneInfo("America/New_York"))

    # Ensure ETF trading only happens during market hours (9:30 AM - 4:00 PM ET)
    # if now.hour < 9 or now.hour >= 16:
    #     return  

    for symbol in etfs_VIS:
        df = get_real_time_market_data(symbol)
        if df is None or df.empty:
            continue

        # Get current price information
        market_open_price = df["open"].iloc[-1] if now.hour == 9 and now.minute == 30 else None
        current_price = df["close"].iloc[-1]

        # # Initialize first purchase logic at market open
        # if symbol not in last_buy_price:
        #     first_buy_price = market_open_price if market_open_price is not None else current_price
        #     last_buy_price[symbol] = first_buy_price  # Store first buy price
        #     holding_etf[symbol] = int(initial_buy_notional / first_buy_price)  # Ensure integer shares
        #     cash_etf[symbol] = -initial_buy_notional

        #     # Execute market order to buy ETFs
        #     if holding_etf[symbol] > 0:
        #         place_order(symbol, qty=holding_etf[symbol], side="buy")
        #         print(f"{symbol} First Buy: ${initial_buy_notional}, Price: {first_buy_price:.2f}")

        # Calculate price change from last purchase price
        
        price_change = (current_price / 252.17) - 1
        holding_etf_keep_VIS[symbol]=79
        cash_etf[symbol]=-19921.43
        
        # Selling Logic
        if holding_etf_keep_VIS[symbol] > 0:
            if price_change >= threshold_sell_all_VIS:
                # Sell all holdings
                place_order(symbol, qty=holding_etf_keep_VIS[symbol], side="sell")
                cash_etf[symbol] += holding_etf_keep_VIS[symbol] * current_price
                print(f"Sold ALL {symbol} at {current_price:.2f}, Total Cash: {cash_etf[symbol]:.2f}")
                holding_etf_keep_VIS[symbol] = 0
            elif price_change >= threshold_sell_40_VIS:
                # Sell 40% of holdings
                sell_qty = int(holding_etf_keep_VIS[symbol] * 0.4)
                if sell_qty > 0:
                    place_order(symbol, qty=sell_qty, side="sell")
                    cash_etf[symbol] += sell_qty * current_price
                    holding_etf_keep_VIS[symbol] -= sell_qty
                    print(f"Sold 40% {symbol} at {current_price:.2f}, Remaining: {holding_etf_keep_VIS[symbol]} shares")
            elif price_change >= threshold_sell_20_VIS:
                # Sell 20% of holdings
                sell_qty = int(holding_etf_keep_VIS[symbol] * 0.2)
                if sell_qty > 0:
                    place_order(symbol, qty=sell_qty, side="sell")
                    cash_etf[symbol] += sell_qty * current_price
                    holding_etf_keep_VIS[symbol] -= sell_qty
                    print(f"Sold 20% {symbol} at {current_price:.2f}, Remaining: {holding_etf_keep_VIS[symbol]} shares")


In [105]:
etfs_keep = ["SPY"]
holding_etf_keep_keep={symbol: 0.0 for symbol in etfs_VIS}

In [106]:
def execute_etf_trading_keep():
    global last_buy_price, holding_etf_keep_keep, cash_etf

    now = datetime.now(ZoneInfo("America/New_York"))

    # Ensure ETF trading only happens during market hours (9:30 AM - 4:00 PM ET)
    # if now.hour < 9 or now.hour >= 16:
    #     return  

    for symbol in etfs_VIS:
        df = get_real_time_market_data(symbol)
        if df is None or df.empty:
            continue

        # Get current price information
        market_open_price = df["open"].iloc[-1] if now.hour == 9 and now.minute == 30 else None
        current_price = df["close"].iloc[-1]

        # # Initialize first purchase logic at market open
        # if symbol not in last_buy_price:
        #     first_buy_price = market_open_price if market_open_price is not None else current_price
        #     last_buy_price[symbol] = first_buy_price  # Store first buy price
        #     holding_etf[symbol] = int(initial_buy_notional / first_buy_price)  # Ensure integer shares
        #     cash_etf[symbol] = -initial_buy_notional

        #     # Execute market order to buy ETFs
        #     if holding_etf[symbol] > 0:
        #         place_order(symbol, qty=holding_etf[symbol], side="buy")
        #         print(f"{symbol} First Buy: ${initial_buy_notional}, Price: {first_buy_price:.2f}")

        # Calculate price change from last purchase price
        
        price_change = (current_price / 564.47) - 1
        holding_etf_keep_keep[symbol]=35
        cash_etf[symbol]=-19737.20
        
        # Selling Logic
        if holding_etf_keep_keep[symbol] > 0:
            if price_change >= threshold_sell_all_VIS:
                # Sell all holdings
                place_order(symbol, qty=holding_etf_keep_keep[symbol], side="sell")
                cash_etf[symbol] += holding_etf_keep_keep[symbol] * current_price
                print(f"Sold ALL {symbol} at {current_price:.2f}, Total Cash: {cash_etf[symbol]:.2f}")
                holding_etf_keep_keep[symbol] = 0
            elif price_change >= threshold_sell_40_VIS:
                # Sell 40% of holdings
                sell_qty = int(holding_etf_keep_keep[symbol] * 0.4)
                if sell_qty > 0:
                    place_order(symbol, qty=sell_qty, side="sell")
                    cash_etf[symbol] += sell_qty * current_price
                    holding_etf_keep_keep[symbol] -= sell_qty
                    print(f"Sold 40% {symbol} at {current_price:.2f}, Remaining: {holding_etf_keep_keep[symbol]} shares")
            elif price_change >= threshold_sell_20_VIS:
                # Sell 20% of holdings
                sell_qty = int(holding_etf_keep_keep[symbol] * 0.2)
                if sell_qty > 0:
                    place_order(symbol, qty=sell_qty, side="sell")
                    cash_etf[symbol] += sell_qty * current_price
                    holding_etf_keep_keep[symbol] -= sell_qty
                    print(f"Sold 20% {symbol} at {current_price:.2f}, Remaining: {holding_etf_keep_keep[symbol]} shares")
                    
        # Rebuy Logic (if price drops 0.5% below the last purchase price)
        if holding_etf_keep_keep[symbol] == 0 and price_change <= threshold_rebuy:
            rebuy_shares = int(rebuy_notional / current_price)
            if rebuy_shares > 0:
                place_order(symbol, qty=rebuy_shares, side="buy")
                holding_etf_keep_keep[symbol] = rebuy_shares
                cash_etf[symbol] -= rebuy_notional
                last_buy_price[symbol] = current_price  # Update last purchase price
                print(f"Rebought {symbol}: {rebuy_shares} shares at {current_price:.2f}, Total Cash: {cash_etf[symbol]:.2f}")


## Stock Strategy

In [107]:
import pandas as pd
import numpy as np
import scipy.signal
import time
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import joblib
import pytz
from datetime import datetime
from datetime import tzinfo

In [108]:
# Initialize portfolio for stock "NVDA"
portfolio_cash = 10000  # Starting capital
holdings = 0
active_trade = None  # To track open trades
trade_times_dict = {"buy_times": [], "sell_times": [], "half_sell_times": []}  
symbol = "NVDA"
active_trades = {}

In [109]:
# Load the MinMaxScaler for "NVDA"
scaler = joblib.load("/Users/dingyiling/Desktop/RealTime_Group5_TradingCompetition/scaler_NVDA.pkl")
model = tf.keras.models.load_model("/Users/dingyiling/Desktop/RealTime_Group5_TradingCompetition/lstm_local_extrema_NVDA.keras")

/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.6.1 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [110]:
# Set the timezone for Chicago (CST/CDT)
CHICAGO_TZ = pytz.timezone("America/New_York")

In [111]:
# Compute technical indicators
def calculate_technical_indicators(df):
    df["SMA_6H"] = df["close"].rolling(window=360, min_periods=1).mean()
    df["EWMA_6H"] = df["close"].ewm(span=360, adjust=False).mean()
    df["momentum_2H"] = df["close"].diff(periods=120)

    delta = df["close"].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=360, min_periods=1).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=360, min_periods=1).mean()
    rs = gain / (loss + 1e-9)
    df["RSI_6H"] = 100 - (100 / (1 + rs))

    df.fillna(method="ffill", inplace=True)
    df.fillna(method="bfill", inplace=True)
    return df

In [112]:
# Prepare and scale data
def prepare_and_scale(df, scaler, features):
    df = calculate_technical_indicators(df)
    df.dropna(inplace=True)
    df[features] = scaler.fit_transform(df[features])
    return df

In [113]:
# Predict next 30 minutes prices
def predict_next_30_minutes(df, model, scaler, features):
    if len(df) < 120:
        return None
    data_3d = df[features].iloc[-120:].values.reshape((1, 120, len(features)))
    y_pred_scaled = model.predict(data_3d)
    
    y_pred_actual = []
    for i in range(30):
        dummy_row = np.zeros((1, len(features)))
        dummy_row[0, 0] = y_pred_scaled[0, i]
        inv = scaler.inverse_transform(dummy_row)
        y_pred_actual.append(inv[0, 0])

    return np.array(y_pred_actual)

In [114]:
# Identify buy/sell signals
def find_trade_times(predicted_prices):
    """
    Identifies the absolute minimum (buy) and maximum (sell) prices in the 30-minute prediction window.
    """
    if predicted_prices is None or len(predicted_prices) == 0:
        return [], []

    # Find the index of the absolute minimum price (Best Buy Time)
    min_index = np.argmin(predicted_prices)

    # Find the index of the absolute maximum price (Best Sell Time)
    max_index = np.argmax(predicted_prices)

    return [min_index], [max_index]  # Convert to list format for consistency

In [115]:
# Predict and store predictions every 30 minutes
def predict_and_store():
    """
    Predicts stock prices for the next 30 minutes, stores them, and prints buy/sell signals immediately.
    """
    global latest_predictions, buy_times, sell_times, trade_times_dict
    now = datetime.now()

    # Fetch real-time market data
    df = get_real_time_market_data(symbol)

    # Scale the data
    df_scaled = prepare_and_scale(df, scaler, ["close", "SMA_6H", "EWMA_6H", "momentum_2H", "RSI_6H", "vwap"])

    # Make predictions
    latest_predictions = predict_next_30_minutes(df_scaled, model, scaler, ["close", "SMA_6H", "EWMA_6H", "momentum_2H", "RSI_6H", "vwap"])

    if latest_predictions is not None:
        print(f"\nUpdated Prediction at {now.strftime('%H:%M')}:")
        print(latest_predictions)

        # Find absolute buy and sell times (min and max price index)
        buy_times, sell_times = find_trade_times(latest_predictions)

        # Ensure we have valid buy/sell signals before proceeding
        if buy_times and sell_times:
            # Convert index positions to actual timestamps
            buy_time_actual = now + timedelta(minutes=int(buy_times[0]))
            sell_time_actual = now + timedelta(minutes=int(sell_times[0]))

            # Calculate half-sell time (midpoint between buy and sell)
            half_sell_time_actual = buy_time_actual + (sell_time_actual - buy_time_actual) / 2

            trade_times_dict["buy_times"] = [buy_time_actual.strftime('%H:%M')]
            trade_times_dict["sell_times"] = [sell_time_actual.strftime('%H:%M')]
            trade_times_dict["half_sell_times"] = [half_sell_time_actual.strftime('%H:%M')]

            # Print Buy/Sell/Half-Sell times immediately
            print("\nTrading Signals (Based on Prediction):")
            print(f"Buy Time: {trade_times_dict['buy_times']}")
            print(f"Sell Time: {trade_times_dict['sell_times']}")
            print(f"Half-Sell Time: {trade_times_dict['half_sell_times']}")
        else:
            print("No valid buy/sell signals found in predictions.")

In [116]:
predict_and_store()

/var/folders/1c/4396k1751xz9sj3p_s1qm1wc0000gn/T/ipykernel_39763/2122819317.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", inplace=True)
/var/folders/1c/4396k1751xz9sj3p_s1qm1wc0000gn/T/ipykernel_39763/2122819317.py:14: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="bfill", inplace=True)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 567ms/step

Updated Prediction at 23:50:
[105.01196349 105.10071639 104.97713445 105.09647695 105.09954853
 105.147014   105.12844169 105.19164729 105.11371051 105.11278646
 105.2941414  105.31617452 105.30220248 105.38424554 105.26451166
 105.48459268 105.48115707 105.56786962 105.73928716 105.55481871
 105.71599656 105.44704155 105.68931239 105.772899   105.56302939
 105.46077837 105.70966424 105.87879185 105.79841548 105.78548959]

Trading Signals (Based on Prediction):
Buy Time: ['23:52']
Sell Time: ['00:17']
Half-Sell Time: ['00:04']


In [117]:
# Risk management parameters
STOP_LOSS_THRESHOLD = 0.99  # Sell if price falls 1% below entry
TRAILING_STOP_THRESHOLD = 0.995  # Sell if price drops 0.5% from peak
TRADE_RISK_PERCENTAGE = 0.05  # Risk 10% of available cash per trade

# Set the timezone for Chicago (CST/CDT)
CHICAGO_TZ = pytz.timezone("America/New_York")
TRADING_TIMEZONE = ZoneInfo("America/New_York")

In [ ]:
def execute_stock_trading():
    global trade_times_dict, symbol
    
    print("\nTrading Execution Start")
    print(f"Current Trade Signals: {trade_times_dict}")
    
    now = datetime.now(ZoneInfo("America/Chicago")).replace(second=0, microsecond=0)
    print(f" Current Time (NY Time): {now}")

    if not trade_times_dict or not trade_times_dict["buy_times"] or not trade_times_dict["sell_times"]:
        print(" No trading signals available. Skipping trade execution.")
        return

    # Get latest real-time market data
    df = get_real_time_market_data(symbol)
    current_price = df["close"].iloc[-1]  # Latest market price
    print(f"Current Market Price: {current_price}")

    buy_time = datetime.strptime(trade_times_dict["buy_times"][0], "%H:%M").replace(
        year=now.year, month=now.month, day=now.day, tzinfo=TRADING_TIMEZONE
    )

    sell_time = datetime.strptime(trade_times_dict["sell_times"][0], "%H:%M").replace(
        year=now.year, month=now.month, day=now.day, tzinfo=TRADING_TIMEZONE
    )

    half_sell_time = datetime.strptime(trade_times_dict["half_sell_times"][0], "%H:%M").replace(
        year=now.year, month=now.month, day=now.day, tzinfo=TRADING_TIMEZONE
    )

    # Debugging Buy Order Execution
    print(f" Checking Buy Time: {buy_time} (Expected Execution Time)")
    if now == buy_time:
        print(f" Match Found! Executing Buy Order at {now}")

        # Calculate order quantity
        buy_amount = portfolio_cash[symbol] * TRADE_RISK_PERCENTAGE  # 20% of available cash
        qty_to_buy = int(buy_amount / current_price)

        print(f"Portfolio Cash: {portfolio_cash[symbol]:.2f}, Buy Amount: {buy_amount:.2f}")
        print(f" Quantity to Buy: {qty_to_buy}, Required Cash: {qty_to_buy * current_price:.2f}")

        if qty_to_buy > 0:
            place_order(symbol, qty=qty_to_buy, side="buy")
            holdings[symbol] += qty_to_buy
            active_trades[symbol] = {
                "buy_price": current_price,
                "highest_price": current_price,
                "trade_time": now
            }
            print(f"{symbol}: Bought {qty_to_buy} shares at {current_price:.2f}")

    # Debugging Sell Order Execution
    print(f"Checking Sell Time: {sell_time} (Expected Execution Time)")
    if now == sell_time and holdings[symbol] > 0:
        print(f" Match Found! Executing Sell Order at {now}")
        place_order(symbol, qty=holdings[symbol], side="sell")
        portfolio_cash[symbol] += holdings[symbol] * current_price
        holdings[symbol] = 0  # Exit position
        print(f" {symbol}: Sold ALL at {current_price:.2f}")

    # Debugging Half-Sell Order Execution
    print(f"Checking Half-Sell Time: {half_sell_time} (Expected Execution Time)")
    if now == half_sell_time and holdings[symbol] > 0:
        qty_to_sell = int(holdings[symbol] * 0.3)  # Sell 30%
        if qty_to_sell > 0:
            place_order(symbol, qty=qty_to_sell, side="sell")
            holdings[symbol] -= qty_to_sell
            print(f"{symbol}: Sold 30% ({qty_to_sell} shares) at {current_price:.2f}")

    # Risk Management (Stop-Loss & Trailing Stop)
    if symbol in active_trades:
        trade_data = active_trades[symbol]
        highest_price = max(trade_data["highest_price"], current_price)

        # Update highest price seen
        active_trades[symbol]["highest_price"] = highest_price

        # Stop-Loss Check
        if current_price <= trade_data["buy_price"] * STOP_LOSS_THRESHOLD:
            print(f"Stop-loss triggered for {symbol}. Selling all holdings.")
            place_order(symbol, qty=holdings[symbol], side="sell")
            holdings[symbol] = 0
            print(f" {symbol}: Stop-loss triggered. Sold at {current_price:.2f}")

        # Trailing Stop Check
        elif current_price <= highest_price * TRAILING_STOP_THRESHOLD:
            print(f" Trailing stop triggered for {symbol}. Selling all holdings.")
            place_order(symbol, qty=holdings[symbol], side="sell")
            holdings[symbol] = 0
            print(f" {symbol}: Trailing stop triggered. Sold at {current_price:.2f}")

# Strategy Execution

In [119]:
# #Save data to local for future analysis
# schedule.every().day.at("16:00").do(fetch_and_save_data, symbols)


# # Schedule ETFs Trading Execution
# schedule.every().day.at("09:30").do(execute_etf_trading_new)
schedule.every().minute.do(execute_etf_trading_new)
# schedule.every().day.at("09:30").do(execute_etf_trading_keep)
schedule.every().minute.do(execute_etf_trading_keep)
# schedule.every().day.at("09:30").do(execute_etf_trading_VIS)
schedule.every().minute.do(execute_etf_trading_VIS)

# Schedule Stock Trading Execution+ since we need first day data to train the model, so monday we only buy ETFS
schedule.every(30).minutes.do(predict_and_store)
schedule.every().minute.do(execute_stock_trading)

while True:
    schedule.run_pending()
    time.sleep(60)

CME - Order Placed: BUY 19 shares
CME First Buy: $5000.0, Price: 262.28
REGN - Order Placed: BUY 6 shares
REGN First Buy: $5000.0, Price: 745.00

 DEBUG: Trading Execution Start
Current Trade Signals: {'buy_times': ['23:52'], 'sell_times': ['00:17'], 'half_sell_times': ['00:04']}
 Current Time (NY Time): 2025-03-11 00:50:00-04:00
Current Market Price: 104.67
 Checking Buy Time: 2025-03-11 23:52:00-04:00 (Expected Execution Time)
Checking Sell Time: 2025-03-11 00:17:00-04:00 (Expected Execution Time)
Checking Half-Sell Time: 2025-03-11 00:04:00-04:00 (Expected Execution Time)

 DEBUG: Trading Execution Start
Current Trade Signals: {'buy_times': ['23:52'], 'sell_times': ['00:17'], 'half_sell_times': ['00:04']}
 Current Time (NY Time): 2025-03-11 00:50:00-04:00
Current Market Price: 104.67
 Checking Buy Time: 2025-03-11 23:52:00-04:00 (Expected Execution Time)
Checking Sell Time: 2025-03-11 00:17:00-04:00 (Expected Execution Time)
Checking Half-Sell Time: 2025-03-11 00:04:00-04:00 (Expect

KeyboardInterrupt: 